In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time
import scipy.stats as stats
from sklearn.impute import KNNImputer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, classification_report
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv', sep='\t')

In [ ]:
data.head()

#### 1) Business understanding: 
**What we are facing here is a key business question: "To whom should we advertise and which products?"**

To answer this question we have to refer to an unsupervised learning algorithm like K-Mean Clustering. Because we don't already have any classification for the customers at hand. So we have to allow the machine to that for us and then we evaluate it.

#### 2) Data Cleaning:

In [ ]:
data.nunique()

In [ ]:
# Z_CostContact and Z_Revenue don't contribute to our Model. They have only one value in all rows.
data.drop(columns = ['Z_CostContact', 'Z_Revenue'], inplace =True)

In [ ]:
# Checking number of unique categories present in the "Marital_Status"

data['Marital_Status'].value_counts()

* We are grouping 'Married', 'Together' as "relationship"
* Whereas 'Divorced', 'Widow', 'Alone', 'YOLO', 'Absurd' as "Single"

In [ ]:
data['Marital_Status'] = data['Marital_Status'].replace(['Married', 'Together'],'relationship')
data['Marital_Status'] = data['Marital_Status'].replace(['Divorced', 'Widow', 'Alone', 'YOLO', 'Absurd'],'Single')
data['Marital_Status'].value_counts()

In [ ]:
data['Education'].value_counts()

grouping 
* 'PhD', 'Master', '2n Cycle' as '4' 
* 'Basic' as '0'
* 'Graduation', as "2"

In [ ]:
encoding_education = {'Basic':0,'Graduation':2,'2n Cycle':4, 'Master':4, 'PhD':4}

data['Education'] = data['Education'].replace(encoding_education)
data['Education'].value_counts()

* Transforming Dt_customer column to Customer Lifespan
* transforming Year_Birth to Age

In [ ]:
current_time = time.time()
current_year = dt.datetime.fromtimestamp(current_time).year

# Dt_Customer column - first Replace incorrect values with NaN
data['Dt_Customer'] = data['Dt_Customer'].apply(lambda x: x if pd.to_datetime(x, errors='coerce', dayfirst=True) is not pd.NaT else None)

# Remove rows with invalid date values
data.dropna(subset=['Dt_Customer'], inplace = True)

# transforming
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'], dayfirst=True)
data['Dt_Customer'] = data['Dt_Customer'].dt.year
data['Customer_Lifespan'] = current_year - data['Dt_Customer']

# Year_Birth column
data['Age'] = current_year - data['Year_Birth']

data

In [ ]:
data.isnull().sum()

As it is only 24 out of 2240 data points , we can drop it. but i want to impute it using Education and year_birth and income:

In [ ]:
imputer = KNNImputer(n_neighbors=10)

x = data[['Year_Birth','Education', 'Income']]

# now we can Impute our null values:
x_imputed = imputer.fit_transform(x)

# make dataframe
x_imputed = pd.DataFrame(x_imputed, columns = x.columns)

#replace the Income column from x in original 'data':
data['Income'] = x_imputed['Income']

In [ ]:
data.describe()

As we see in min of Year_Birth, we see outlier/-s.

As we see in min and max of Income column, there exists outlier/-s.

As we see in max of MntWines, MntFruits, MntMeatProducts, MntFishProducts, MntSweetProducts, MntGoldProds columns, there exists outlier/-s.

In [ ]:
#Let's see what is going on in Year_Birth, Income, and Minutes in different sections:
sns.set()
fig, axes = plt.subplots(2, 3, figsize=(16,7))

axes[0,0].hist(data['Age'])
axes[0,0].set_title('Age')

axes[0,1].hist(data['Income'])
axes[0,1].set_title('Income')

axes[0,2].hist(data['MntFruits'])
axes[0,2].set_title('Amount Spent On Fruit')

axes[1,0].hist(data['MntMeatProducts'])
axes[1,0].set_title('Amount Spent On Meat')

axes[1,1].hist(data['MntSweetProducts'])
axes[1,1].set_title('Amount Spent On Sweet')

axes[1,2].hist(data['MntWines'])
axes[1,2].set_title('Amount Spent On Wine')

plt.tight_layout()

#### Our guess was true about income , age and MntMeatProds but the other columns are right skewed. not just couple of outliers.

In [ ]:
# removing 0.5% from the upper side of 'Age' column
upper_bound = np.percentile(data['Age'], 99.5)
data = data[data['Age'] <= upper_bound]

# removing 0.5% from the upper side of 'Income' column
upper_bound = np.percentile(data['Income'], 99.5)
data = data[data['Income'] <= upper_bound]

# removing outliers of MntMeat for more that 1200 minutes and MntGold for more than 250 (we will remove 6 rows overall)
data = data[data['MntMeatProducts'] < 1250]
data = data[data['MntGoldProds'] <= 250]

In [ ]:
sns.set()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,3))

ax1.hist(data['Year_Birth'])
ax2.hist(data['Income'])

#### Much Better result just by removing 24 rows equal to 1% of the data.

In [ ]:
# let's see the correlation between the amount spent on different categories:
sns.reset_orig()
plt.figure(figsize=(6,4))

# how spent features are correlated?
corr = data[["MntFruits", "MntMeatProducts", "MntFishProducts", "MntSweetProducts", "MntGoldProds", "MntWines"]].corr(numeric_only = True)

sns.heatmap(corr, cmap = 'Greys', annot=True, annot_kws={'fontsize':5.5})

##### We see a positive correlatoon between the amount spent in different categories. 
##### so for reducing the dimesionality we can add them together for each person as customer lifetime value.

In [ ]:
# reducing dimension by combining some columns:
data['Kids'] = data['Kidhome'] + data['Teenhome']

data['TotalAcceptedCmp'] = data['AcceptedCmp1'] + data['AcceptedCmp2'] + data['AcceptedCmp3'] \
+ data['AcceptedCmp4'] + data['AcceptedCmp5'] + data['Response']

data['Customer_Lifetime_Value'] = data['MntFruits'] + data['MntMeatProducts'] + data['MntFishProducts'] + data['MntSweetProducts'] \
+ data['MntGoldProds'] + data['MntWines']

# Deleting some column to reduce dimension and complexity of model
col_del = ["AcceptedCmp1" , "AcceptedCmp2", "AcceptedCmp3" , "AcceptedCmp4",'Teenhome', 'Kidhome',
           "MntFruits", "MntMeatProducts", "MntFishProducts", "MntSweetProducts", "MntGoldProds", "MntWines",
           "AcceptedCmp5", "Response", "Dt_Customer", "Year_Birth"]
data = data.drop(columns=col_del, axis=1)

# ordering the columns again:
data = data[['ID', 'Age', 'Kids', 'Education', 'Income' , 'Marital_Status', 'TotalAcceptedCmp', 'Customer_Lifespan', 'Recency', 'Customer_Lifetime_Value',
             'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Complain']]
data.head()

#### 3) Exploratory Data Analysis:

In [ ]:
sns.reset_orig()
plt.figure(figsize=(9,7))

# how features are correlated?
corr = data.corr(numeric_only = True)

# removing the upper triangle
mask = np.triu(np.ones_like(corr, dtype = bool))

# color map for better visualization
cmap = sns.color_palette('coolwarm', as_cmap = True)

#
sns.heatmap(corr, cmap = cmap, mask = mask, annot=True, annot_kws={'fontsize':5.5})

In [ ]:
sns.set()

# group the number of web visits by number of kids in Home.
kid_web_corr = data['NumWebVisitsMonth'].groupby(data['Kids']).mean()

# group the income by number of Kids at home.
income_kid_corr = data['Income'].groupby(data['Kids']).mean()

# group the number of web purchases by number of kids at home.
kid_web_purchases_corr = data['NumWebPurchases'].groupby(data['Kids']).mean()

# numbers to visualize
charts_df=[kid_web_corr, kid_web_purchases_corr, income_kid_corr]

# Visualization:
fig, axes = plt.subplots(1, 3,  figsize=(12, 4), facecolor=(0.761, 0.984, 0, 0.25))

# titles
axes[0].set_title('Avg num of Web visits per num of kids', fontsize=10, pad=14)
axes[1].set_title('Avg num of Web purchases per num of kids', fontsize=10, pad=14)
axes[2].set_title('Avg income & number of kids', fontsize=10, pad=14)

# charts
for i in range(0, 3):
    axes[i].tick_params(labelsize=9)
    axes[i].bar(charts_df[i].index, charts_df[i].values, edgecolor='0.6',
                linewidth=0.5, width=0.8, facecolor=(0.761, 0.984, 0, 0.4))
    axes[i].set_xlabel('Number of Kids', fontsize = 9)
    
    x_ticks = np.arange(0, max(charts_df[i].index+1), 1)
    axes[i].set_xticks(x_ticks)


plt.tight_layout(pad=2)

* **More kids = More web visits BUT less purchases.** Not only web purchases , but also store purchases according to correlation Matrix. it makes sense that they visit website to buy online because they have less time to go outside but why do they don't buy?
The answer is in the right chart. Families with less income have more kids. So they visit the website more but they can't usually buy.

* According to correlation matix , **more income = more in store purchases and less web visits.**
* Families with kids, are more interested in buying with discount.

In [ ]:
df = pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv', sep='\t')

# new dataframe just for Campaigns
accepted_campaigns = pd.DataFrame(index = ['Average_Income', 'Total_Num_Families', 'Year_Birth', 'KidHome'], 
                                  columns = ['NotAccepted_Cmp1', 'Accepted_Cmp1', 'NotAccepted_cmp2', 'Accepted_Cmp2', 
                                            'NotAccepted_Cmp3', 'Accepted_Cmp3', 'NotAccepted_cmp4', 'Accepted_Cmp4',
                                            'NotAccepted_Cmp5', 'Accepted_Cmp5'])

# loop for filling the dataframe
for i in range(0, 5):
    j = i * 2
    accepted_campaigns.iloc[0, j:j+2] = round(df['Income'].groupby(df[f'AcceptedCmp{i+1}']).mean()).transpose()
    accepted_campaigns.iloc[1, j:j+2] = round(df['Income'].groupby(df[f'AcceptedCmp{i+1}']).count()).transpose()
    accepted_campaigns.iloc[2, j:j+2] = round(df['Year_Birth'].groupby(df[f'AcceptedCmp{i+1}']).mean()).transpose()
    accepted_campaigns.iloc[3, j:j+2] = round(df['Kidhome'].groupby(df[f'AcceptedCmp{i+1}']).mean(), ndigits=2).transpose()

accepted_campaigns

* Around 7% of the customers accepted each campaign. Campain 2 was the worst with 1.3% acceptance.
* Around 50% of the Families with Kids didn't accept any campaigns except for Campain 3. So surely it has something interesting for them.
* Families with more Income accepted the campaigns.
* From the average Age of the customers, we cannot conclude anything because they are very close. (Correlation Matrix confirms it)

In [ ]:
plt.scatter(data['Income'], data['Customer_Lifetime_Value'])
plt.xlabel('Income')
plt.ylabel('CLV')

In [ ]:
plt.hist(data['Customer_Lifetime_Value'])
plt.xlabel('Amount')
plt.ylabel('Count')

As we can see, Customer Lifetime Value is heavily right skewed. we see Heteroscadastisity. On the other hand these CLV and Income feature have huge variance and they can affect the clustering because K-Mean Clustering is a distance sensitive method.
So the best way to solve this is to transform and Scale all the columns.

In [ ]:
# let's see the influence of log transform:
plt.scatter(np.cbrt(data['Income']), np.log(data['Customer_Lifetime_Value']))
plt.xlabel('Income')
plt.ylabel('CLV')

**Now we have more balanced data.**

In [ ]:
# first using OneHotEncoding to change categories to numerical columns
columns_to_encode = pd.get_dummies(data['Marital_Status'], dtype='int', drop_first= True)

# joining the dummies columns with our  dataframe
data_encoded = pd.concat([data, columns_to_encode], axis = 1)

# drop the original columns
data_encoded.drop(labels = ['Marital_Status', 'ID'], axis = 1, inplace = True)

#### 4) Feature Selection:

in order to use k-means Clustering , we have to reduce the dimension as much as possible. Let's do that using PCA.

In [ ]:
# creating train test data
df_train, df_test = train_test_split(data_encoded, test_size=0.2, random_state = 77)

scaler = StandardScaler()

# train data:
data_scaled = scaler.fit_transform(df_train)
data_train = pd.DataFrame(columns = data_encoded.columns, data=data_scaled)

# test data:
data_scaled_ = scaler.transform(df_test)
data_test = pd.DataFrame(columns = data_encoded.columns, data = data_scaled_)

# I preserve d_train and d_test dataframe for later use
data_train.head(3)

In [ ]:
number_of_components = 2
pca = PCA(n_components= number_of_components)
pca.fit(data_train)

# let's see how much variance do the first 2 components explained
pca.explained_variance_ratio_

As we see the first 2 explained around 42% of the variance which is not very impressive.

PCA Dataframe:

In [ ]:
columns = []
pca_data_train = pca.transform(data_train)
for component in range(0, number_of_components):
    column_name = 'Component_' + str(component + 1)
    columns.append(column_name)
    
pca_df = pd.DataFrame(columns=columns, data=pca_data_train)
pca_df.head(3)

#### Evaluation using Elbow method to find the best number of clusters:

In [ ]:
# elbow mthod:

# Initialize an empty list to store the inertia values
inertia_values = []

# Fit K-Means models for different values of k and calculate inertia
for k in range(2,11):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=77, n_init='auto')  # You can adjust random_state
    kmeans.fit(pca_df)  
    inertia_values.append(kmeans.inertia_)
    
# Plot the Elbow Curve
plt.figure(figsize=(8, 6))
plt.plot(range(2,11), inertia_values, marker='o', linestyle='-', color='b')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()

#### Evaluation using Silhouette score to find the best number of clusters:

In [ ]:
for num_clusters in range(2, 11):  # Evaluate cluster numbers from 2 to 10
    kmeans = KMeans(n_clusters=num_clusters, n_init = 'auto', random_state= 77)
    kmeans.fit(pca_df)
    labels = kmeans.labels_
    silhouette_avg = silhouette_score(pca_df, labels)
    print(f"For {num_clusters} clusters, Silhouette Score:", silhouette_avg)

#### Evaluation using calinski harabasz score to find the best number of clusters:

In [ ]:
for num_clusters in range(2, 11):  # Evaluate cluster numbers from 2 to 10
    kmeans = KMeans(n_clusters=num_clusters, n_init= 'auto', random_state = 77)  # Replace num_clusters with the number of clusters you want to evaluate
    kmeans.fit(pca_df)
    labels = kmeans.labels_

    # Calculate the Calinski-Harabasz score
    calinski_harabasz = calinski_harabasz_score(pca_df, labels)
    print(f"For {num_clusters} clusters, Calinski-Harabasz Score:", calinski_harabasz)

In [ ]:
kmeans = KMeans(n_clusters=3, n_init= 'auto', random_state = 77)
clusters = kmeans.fit_predict(pca_df)
pca_df['Target'] = clusters
pca_df.head(3)

In [ ]:
df_train.reset_index(inplace = True, drop = True)
df_train['Target'] = pca_df['Target']
df_train.head(2)

### Testing our Classification using observation in the dataset:

In [ ]:
# Define the columns you want to visualize
columns_selected = ['Age', 'Kids', 'Income', 'TotalAcceptedCmp', 'Customer_Lifespan', 'Recency',
                    'Customer_Lifetime_Value', 'NumDealsPurchases', 'NumWebPurchases',
                    'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
                    'Complain', 
                    'Education',
                    'relationship']

# Create a 4x4 grid of subplots
fig, axes = plt.subplots(4, 4, figsize=(16, 9))

# Flatten the 2D axes array to iterate over it easily
axes = axes.flatten()

# Iterate through the selected columns and plot each one
for i, column in enumerate(columns_selected):
    # Group data by 'Target' and calculate the mean for the current column
    grouped_data = df_train[column].groupby(by=df_train['Target']).mean()
    
    # Determine the row and column for the current subplot
    row = i // 4
    col = i % 4
    
    # Plot the bar chart in the current subplot
    sns.barplot(x=grouped_data.index, y=grouped_data.values, ax=axes[i], palette='viridis')
    axes[i].set_xlabel('Groups', fontsize=9)
    axes[i].set_ylabel(column, fontsize=9)
    axes[i].tick_params(labelsize=9)
    axes[i].set_xticks(grouped_data.index)
    axes[i].set_title(f'Mean {column} by Target', fontsize=10)

# Adjust layout and display the subplots
plt.tight_layout()
plt.show()


Let's see the differences between the groups and 
### evaluating the classes:

* We don't see a signifact difference between the mean of age in different groups.
* group 0 and 2 have on average more than 1 kid. and group 1 barely have 1 kid. 
* Although group 0 have the same amount of kids as group 2 but they have somehow half the income. So we can guess that they are less educated, they complain more and they buy less than other groups. All of these guesses are proven through the charts. 
* Group 2 has the same kids but more reasonable income, they tend to buy deals more often. They have a better CLV than group 0 but less than group 1. They tend to buy online more than other two groups. probably because they have little time outside the workplace and they want to spend time at home with their kids.
* Group 1 have the best income, they accept the campaigns more often, they have the best CLV, they don't care about the deals, they buy from catalog and from store pretty much, they are educated and they complain rarely. 

#### Let's test the result with another number of components:

In [ ]:
number_of_components = 5
pca = PCA(n_components= number_of_components, random_state=100)
pca.fit(data_train)

# let's see how much variance do the first 2 component's explained
pca.explained_variance_ratio_

In [ ]:
columns = []
pca_data_train = pca.transform(data_train)
for component in range(0, number_of_components):
    column_name = 'Component_' + str(component + 1)
    columns.append(column_name)
    
pca_df = pd.DataFrame(columns=columns, data=pca_data_train)
pca_df

In [ ]:
# elbow mthod:

# Initialize an empty list to store the inertia values
inertia_values = []

# Fit K-Means models for different values of k and calculate inertia
for k in range(2,11):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=77, n_init='auto')  # You can adjust random_state
    kmeans.fit(pca_df)  
    inertia_values.append(kmeans.inertia_)
    
# Plot the Elbow Curve
plt.figure(figsize=(8, 6))
plt.plot(range(2,11), inertia_values, marker='o', linestyle='-', color='b')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()

In [ ]:
for num_clusters in range(2, 11):  # Evaluate cluster numbers from 2 to 10
    kmeans = KMeans(n_clusters=num_clusters, n_init = 'auto', random_state= 77)
    kmeans.fit(pca_df)
    labels = kmeans.labels_
    silhouette_avg = silhouette_score(pca_df, labels)
    print(f"For {num_clusters} clusters, Silhouette Score:", silhouette_avg)

In [ ]:
for num_clusters in range(2, 11):  # Evaluate cluster numbers from 2 to 10
    kmeans = KMeans(n_clusters=num_clusters, n_init= 'auto', random_state = 77)  # Replace num_clusters with the number of clusters you want to evaluate
    kmeans.fit(pca_df)
    labels = kmeans.labels_

    # Calculate the Calinski-Harabasz score
    calinski_harabasz = calinski_harabasz_score(pca_df, labels)
    print(f"For {num_clusters} clusters, Calinski-Harabasz Score:", calinski_harabasz)

##### With 5 components the best number of clusters is 2 but it gives us less detailed information about the customers. so we continue using 3 clusters.

In [ ]:
kmeans = KMeans(n_clusters=3, n_init= 'auto', random_state = 77)
clusters = kmeans.fit_predict(pca_df)
pca_df['Target'] = clusters
pca_df.head(2)

In [ ]:
df_train.reset_index(inplace = True, drop = True)
df_train['Target'] = pca_df['Target']
df_train.head(2)

### Let's classify our testing data and compare it with knn classifier.

In [ ]:
# transforming the features of our test dataset using PCA model that we trained
pca_df_test_ = pca.transform(data_test)
pca_df_test = pd.DataFrame(columns=columns, data=pca_df_test_)

# predicting our test data using kmeans
clusters = kmeans.predict(pca_df_test)
pca_df_test['Target'] = clusters

# concat Target columns with original data
df_test.reset_index(drop = True, inplace = True)
df_test['Target'] = pca_df_test['Target']
df_test.head(2)

In [ ]:
# which features contribute more to the variance according to Principle component analysis:
pca_components = pd.DataFrame(data = pca.components_, columns = data_train.columns)

# we want to see all the columns
pd.set_option('display.max_columns', None)

pca_components

In [ ]:
# loop to iterate through all the pca_components dataframe and printing the feature names which the values > 0.2 which are more important
cols = []
for i in range(0, len(pca_components.columns)):
    [cols.append(pca_components.columns[i]) for x in pca_components.iloc[: , i].values if x > 0.3 and
     pca_components.columns[i] not in cols]
    
print(cols)

In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=5)

x_train = df_train[cols]
y_train = df_train['Target']

clf_knn.fit(x_train, y_train)

In [ ]:
y_pred = clf_knn.predict(df_test[cols])
y_true = df_test['Target']

print(classification_report(y_true, y_pred))

Although i have trained the KNN by the original data, it gave me a pretty good result. The result would be much higher if i would have trained the model using the scaled data. But this result is more reliable because it is not overfitted. I should also mention that this evaluation is just a metric we can look into but observation in the data is more important because KMeans is an unsupervised machine learning model and if it poorly categorize the data, this classification report is also unusable.  